# Dashboard Interactivo - Precios de Ganado en Panamá

Este notebook proporciona un análisis estadístico exploratorio (EDA) de los precios de ganado en Panamá, con visualizaciones interactivas y capacidad de filtrado y descarga de datos.

In [1]:
# Importar bibliotecas
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sqlite3
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

print("✓ Bibliotecas cargadas exitosamente")

ModuleNotFoundError: No module named 'pandas'

## 1. Cargar Datos

In [ ]:
# Función para cargar datos desde CSV o SQLite
def load_data():
    """Carga datos desde CSV o SQLite"""
    import os

    # Intentar cargar desde archivo sin outliers primero
    csv_clean_path = 'data/precios_ganado_sin_outliers.csv'
    csv_path = 'data/precios_ganado_clean.csv'
    db_path = 'data/precios_ganado.db'

    if os.path.exists(csv_clean_path):
        print(f"Cargando datos limpios (sin outliers): {csv_clean_path}")
        df = pd.read_csv(csv_clean_path)
    elif os.path.exists(csv_path):
        print(f"Cargando datos desde CSV: {csv_path}")
        df = pd.read_csv(csv_path)
    elif os.path.exists(db_path):
        print(f"Cargando datos desde SQLite: {db_path}")
        conn = sqlite3.connect(db_path)
        df = pd.read_sql_query("SELECT * FROM precios_ganado", conn)
        conn.close()
    else:
        raise FileNotFoundError("No se encontró el archivo de datos. Ejecute primero pdf_extractor_v2.py")

    # Convertir columnas de fecha
    if 'fecha_desde' in df.columns:
        df['fecha_desde'] = pd.to_datetime(df['fecha_desde'], format='%d/%m/%Y', errors='coerce')
    if 'fecha_hasta' in df.columns:
        df['fecha_hasta'] = pd.to_datetime(df['fecha_hasta'], format='%d/%m/%Y', errors='coerce')

    # Crear columna de fecha promedio para análisis
    if 'fecha_desde' in df.columns and 'fecha_hasta' in df.columns:
        df['fecha'] = df[['fecha_desde', 'fecha_hasta']].mean(axis=1)

    # Asegurar que precio sea numérico
    df['precio'] = pd.to_numeric(df['precio'], errors='coerce')

    # Eliminar filas con precios nulos
    df = df.dropna(subset=['precio'])

    print(f"✓ Datos cargados: {len(df)} registros")
    if 'fecha' in df.columns and not df['fecha'].isna().all():
        print(f"  - Rango de fechas: {df['fecha'].min().date()} a {df['fecha'].max().date()}")
    print(f"  - Lugares: {df['lugar'].nunique()}")
    print(f"  - Categorías: {df['categoria'].nunique()}")
    print(f"  - Rango de precios: ${df['precio'].min():.2f} - ${df['precio'].max():.2f}")

    return df

# Cargar datos
df_original = load_data()
df = df_original.copy()

## 2. Vista Previa de los Datos

In [ ]:
# Mostrar primeras filas
display(HTML("<h3>Primeras filas del dataset</h3>"))
display(df.head(10))

# Información del dataset
display(HTML("<h3>Información del dataset</h3>"))
print(f"Dimensiones: {df.shape[0]} filas x {df.shape[1]} columnas")
print(f"\nColumnas: {list(df.columns)}")
print(f"\nTipos de datos:")
print(df.dtypes)

## 3. Estadísticas Descriptivas

In [ ]:
# Estadísticas generales
display(HTML("<h3>Estadísticas Descriptivas de Precios</h3>"))
display(df['precio'].describe())

# Top 10 lugares con más registros
display(HTML("<h3>Top 10 Lugares con Más Registros</h3>"))
display(df['lugar'].value_counts().head(10))

# Top 10 categorías con más registros
display(HTML("<h3>Top 10 Categorías con Más Registros</h3>"))
display(df['categoria'].value_counts().head(10))

## 4. Controles Interactivos para Filtrado

In [ ]:
# Crear widgets de filtrado
fecha_inicio = widgets.DatePicker(
    description='Fecha Inicio:',
    value=df['fecha'].min().date() if not df.empty else datetime.now().date(),
    style={'description_width': 'initial'}
)

fecha_fin = widgets.DatePicker(
    description='Fecha Fin:',
    value=df['fecha'].max().date() if not df.empty else datetime.now().date(),
    style={'description_width': 'initial'}
)

lugares_list = ['Todos'] + sorted(df['lugar'].unique().tolist())
lugar_selector = widgets.SelectMultiple(
    options=lugares_list,
    value=['Todos'],
    description='Lugares:',
    rows=5,
    style={'description_width': 'initial'}
)

categorias_list = ['Todas'] + sorted(df['categoria'].unique().tolist())
categoria_selector = widgets.SelectMultiple(
    options=categorias_list,
    value=['Todas'],
    description='Categorías:',
    rows=5,
    style={'description_width': 'initial'}
)

precio_min = widgets.FloatText(
    value=df['precio'].min(),
    description='Precio Mín:',
    style={'description_width': 'initial'}
)

precio_max = widgets.FloatText(
    value=df['precio'].max(),
    description='Precio Máx:',
    style={'description_width': 'initial'}
)

btn_filtrar = widgets.Button(
    description='Aplicar Filtros',
    button_style='primary',
    icon='filter'
)

btn_reset = widgets.Button(
    description='Resetear Filtros',
    button_style='warning',
    icon='refresh'
)

output_filtros = widgets.Output()

# Variable global para datos filtrados
df_filtered = df.copy()

def aplicar_filtros(b):
    global df_filtered
    with output_filtros:
        output_filtros.clear_output()
        df_filtered = df_original.copy()
        
        # Filtro de fechas
        if fecha_inicio.value:
            df_filtered = df_filtered[df_filtered['fecha'] >= pd.to_datetime(fecha_inicio.value)]
        if fecha_fin.value:
            df_filtered = df_filtered[df_filtered['fecha'] <= pd.to_datetime(fecha_fin.value)]
        
        # Filtro de lugares
        if 'Todos' not in lugar_selector.value:
            df_filtered = df_filtered[df_filtered['lugar'].isin(lugar_selector.value)]
        
        # Filtro de categorías
        if 'Todas' not in categoria_selector.value:
            df_filtered = df_filtered[df_filtered['categoria'].isin(categoria_selector.value)]
        
        # Filtro de precios
        df_filtered = df_filtered[
            (df_filtered['precio'] >= precio_min.value) & 
            (df_filtered['precio'] <= precio_max.value)
        ]
        
        print(f"✓ Filtros aplicados: {len(df_filtered)} registros seleccionados")

def resetear_filtros(b):
    global df_filtered
    with output_filtros:
        output_filtros.clear_output()
        df_filtered = df_original.copy()
        fecha_inicio.value = df_original['fecha'].min().date()
        fecha_fin.value = df_original['fecha'].max().date()
        lugar_selector.value = ['Todos']
        categoria_selector.value = ['Todas']
        precio_min.value = df_original['precio'].min()
        precio_max.value = df_original['precio'].max()
        print("✓ Filtros reseteados")

btn_filtrar.on_click(aplicar_filtros)
btn_reset.on_click(resetear_filtros)

# Layout de controles
display(HTML("<h3>Panel de Control - Filtros</h3>"))
display(widgets.HBox([
    widgets.VBox([fecha_inicio, fecha_fin, precio_min, precio_max]),
    widgets.VBox([lugar_selector]),
    widgets.VBox([categoria_selector])
]))
display(widgets.HBox([btn_filtrar, btn_reset]))
display(output_filtros)

## 5. Visualizaciones Interactivas

In [ ]:
# 5.1 Evolución temporal de precios
def plot_evolucion_temporal():
    df_temp = df_filtered.groupby('fecha')['precio'].agg(['mean', 'min', 'max']).reset_index()
    
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=df_temp['fecha'],
        y=df_temp['mean'],
        mode='lines+markers',
        name='Precio Promedio',
        line=dict(color='blue', width=2)
    ))
    
    fig.add_trace(go.Scatter(
        x=df_temp['fecha'],
        y=df_temp['max'],
        mode='lines',
        name='Precio Máximo',
        line=dict(color='red', width=1, dash='dash')
    ))
    
    fig.add_trace(go.Scatter(
        x=df_temp['fecha'],
        y=df_temp['min'],
        mode='lines',
        name='Precio Mínimo',
        line=dict(color='green', width=1, dash='dash')
    ))
    
    fig.update_layout(
        title='Evolución Temporal de Precios de Ganado',
        xaxis_title='Fecha',
        yaxis_title='Precio (B/.)',
        hovermode='x unified',
        height=500
    )
    
    return fig

display(HTML("<h3>5.1 Evolución Temporal de Precios</h3>"))
fig1 = plot_evolucion_temporal()
fig1.show()

In [ ]:
# 5.2 Comparación de precios por lugar
def plot_precios_por_lugar():
    df_lugar = df_filtered.groupby('lugar')['precio'].agg(['mean', 'count']).reset_index()
    df_lugar = df_lugar.sort_values('mean', ascending=False).head(15)
    
    fig = px.bar(
        df_lugar,
        x='lugar',
        y='mean',
        text='count',
        title='Top 15 Lugares por Precio Promedio',
        labels={'mean': 'Precio Promedio (B/.)', 'lugar': 'Lugar', 'count': 'Registros'},
        color='mean',
        color_continuous_scale='Viridis'
    )
    
    fig.update_traces(texttemplate='%{text} registros', textposition='outside')
    fig.update_layout(height=500, xaxis_tickangle=-45)
    
    return fig

display(HTML("<h3>5.2 Comparación de Precios por Lugar</h3>"))
fig2 = plot_precios_por_lugar()
fig2.show()

In [ ]:
# 5.3 Comparación de precios por categoría
def plot_precios_por_categoria():
    df_cat = df_filtered.groupby('categoria')['precio'].agg(['mean', 'count']).reset_index()
    df_cat = df_cat.sort_values('mean', ascending=False).head(15)
    
    fig = px.bar(
        df_cat,
        x='categoria',
        y='mean',
        text='count',
        title='Top 15 Categorías por Precio Promedio',
        labels={'mean': 'Precio Promedio (B/.)', 'categoria': 'Categoría', 'count': 'Registros'},
        color='mean',
        color_continuous_scale='Plasma'
    )
    
    fig.update_traces(texttemplate='%{text} registros', textposition='outside')
    fig.update_layout(height=500, xaxis_tickangle=-45)
    
    return fig

display(HTML("<h3>5.3 Comparación de Precios por Categoría</h3>"))
fig3 = plot_precios_por_categoria()
fig3.show()

In [ ]:
# 5.4 Box plot de precios por categoría
def plot_boxplot_categoria():
    # Tomar top 10 categorías por número de registros
    top_cats = df_filtered['categoria'].value_counts().head(10).index
    df_box = df_filtered[df_filtered['categoria'].isin(top_cats)]
    
    fig = px.box(
        df_box,
        x='categoria',
        y='precio',
        title='Distribución de Precios por Categoría (Top 10)',
        labels={'precio': 'Precio (B/.)', 'categoria': 'Categoría'},
        color='categoria'
    )
    
    fig.update_layout(height=500, xaxis_tickangle=-45, showlegend=False)
    
    return fig

display(HTML("<h3>5.4 Distribución de Precios por Categoría</h3>"))
fig4 = plot_boxplot_categoria()
fig4.show()

In [ ]:
# 5.5 Heatmap de precios promedio (Lugar vs Categoría)
def plot_heatmap():
    # Tomar top 10 lugares y categorías
    top_lugares = df_filtered['lugar'].value_counts().head(10).index
    top_categorias = df_filtered['categoria'].value_counts().head(10).index
    
    df_heat = df_filtered[
        (df_filtered['lugar'].isin(top_lugares)) &
        (df_filtered['categoria'].isin(top_categorias))
    ]
    
    pivot_table = df_heat.pivot_table(
        values='precio',
        index='categoria',
        columns='lugar',
        aggfunc='mean'
    )
    
    fig = px.imshow(
        pivot_table,
        title='Mapa de Calor: Precio Promedio por Lugar y Categoría',
        labels=dict(x="Lugar", y="Categoría", color="Precio (B/.)"),
        aspect="auto",
        color_continuous_scale='RdYlGn_r'
    )
    
    fig.update_layout(height=600)
    
    return fig

display(HTML("<h3>5.5 Mapa de Calor: Precios por Lugar y Categoría</h3>"))
fig5 = plot_heatmap()
fig5.show()

In [ ]:
# 5.6 Histograma de distribución de precios
def plot_histogram():
    fig = px.histogram(
        df_filtered,
        x='precio',
        nbins=50,
        title='Distribución de Precios de Ganado',
        labels={'precio': 'Precio (B/.)', 'count': 'Frecuencia'},
        marginal='box'
    )
    
    fig.update_layout(height=500)
    
    return fig

display(HTML("<h3>5.6 Distribución de Precios</h3>"))
fig6 = plot_histogram()
fig6.show()

## 6. Análisis Estadístico de Datos Filtrados

In [ ]:
def mostrar_estadisticas():
    if len(df_filtered) == 0:
        print("No hay datos con los filtros aplicados")
        return
    
    display(HTML("<h3>Estadísticas de Datos Filtrados</h3>"))
    
    print(f"Total de registros: {len(df_filtered)}")
    print(f"\nRango de fechas: {df_filtered['fecha'].min().date()} a {df_filtered['fecha'].max().date()}")
    print(f"\nEstadísticas de precios:")
    print(f"  - Precio mínimo: B/. {df_filtered['precio'].min():.2f}")
    print(f"  - Precio máximo: B/. {df_filtered['precio'].max():.2f}")
    print(f"  - Precio promedio: B/. {df_filtered['precio'].mean():.2f}")
    print(f"  - Mediana: B/. {df_filtered['precio'].median():.2f}")
    print(f"  - Desviación estándar: B/. {df_filtered['precio'].std():.2f}")
    print(f"\nLugares únicos: {df_filtered['lugar'].nunique()}")
    print(f"Categorías únicas: {df_filtered['categoria'].nunique()}")
    
    # Top 5 lugares
    print(f"\nTop 5 lugares por precio promedio:")
    top_lugares = df_filtered.groupby('lugar')['precio'].mean().sort_values(ascending=False).head(5)
    for lugar, precio in top_lugares.items():
        print(f"  - {lugar}: B/. {precio:.2f}")
    
    # Top 5 categorías
    print(f"\nTop 5 categorías por precio promedio:")
    top_cats = df_filtered.groupby('categoria')['precio'].mean().sort_values(ascending=False).head(5)
    for cat, precio in top_cats.items():
        print(f"  - {cat}: B/. {precio:.2f}")

mostrar_estadisticas()

## 7. Exportar Datos Filtrados

In [ ]:
# Botones de exportación
btn_export_csv = widgets.Button(
    description='Exportar a CSV',
    button_style='success',
    icon='download'
)

btn_export_excel = widgets.Button(
    description='Exportar a Excel',
    button_style='success',
    icon='download'
)

output_export = widgets.Output()

def exportar_csv(b):
    with output_export:
        output_export.clear_output()
        filename = f"datos_filtrados_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        df_filtered.to_csv(filename, index=False, encoding='utf-8')
        print(f"✓ Datos exportados a: {filename}")
        print(f"  - Registros: {len(df_filtered)}")

def exportar_excel(b):
    with output_export:
        output_export.clear_output()
        filename = f"datos_filtrados_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
        df_filtered.to_excel(filename, index=False, engine='openpyxl')
        print(f"✓ Datos exportados a: {filename}")
        print(f"  - Registros: {len(df_filtered)}")

btn_export_csv.on_click(exportar_csv)
btn_export_excel.on_click(exportar_excel)

display(HTML("<h3>Exportar Datos Filtrados</h3>"))
display(widgets.HBox([btn_export_csv, btn_export_excel]))
display(output_export)

## 8. Tabla de Datos Filtrados

In [ ]:
display(HTML("<h3>Vista de Datos Filtrados (últimos 100 registros)</h3>"))
display(df_filtered.tail(100))

## 9. Análisis de Tendencias

In [ ]:
# Análisis de tendencia mensual
def plot_tendencia_mensual():
    df_monthly = df_filtered.copy()
    df_monthly['mes'] = df_monthly['fecha'].dt.to_period('M')
    
    df_agg = df_monthly.groupby('mes')['precio'].agg(['mean', 'count']).reset_index()
    df_agg['mes'] = df_agg['mes'].astype(str)
    
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=('Precio Promedio Mensual', 'Número de Registros por Mes'),
        vertical_spacing=0.15
    )
    
    fig.add_trace(
        go.Scatter(
            x=df_agg['mes'],
            y=df_agg['mean'],
            mode='lines+markers',
            name='Precio Promedio',
            line=dict(color='blue', width=2)
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Bar(
            x=df_agg['mes'],
            y=df_agg['count'],
            name='Registros',
            marker_color='lightblue'
        ),
        row=2, col=1
    )
    
    fig.update_xaxes(title_text="Mes", row=2, col=1)
    fig.update_yaxes(title_text="Precio (B/.)", row=1, col=1)
    fig.update_yaxes(title_text="Registros", row=2, col=1)
    
    fig.update_layout(height=700, title_text="Análisis de Tendencias Mensuales", showlegend=False)
    
    return fig

display(HTML("<h3>9.1 Tendencias Mensuales</h3>"))
fig7 = plot_tendencia_mensual()
fig7.show()

## 10. Resumen Final

In [ ]:
display(HTML("<h2 style='color: green;'>Dashboard Completado</h2>"))
display(HTML("""
<div style='background-color: #f0f0f0; padding: 20px; border-radius: 10px;'>
<h3>Instrucciones de Uso:</h3>
<ol>
    <li>Use el <b>Panel de Control</b> (Sección 4) para filtrar datos por fecha, lugar, categoría y rango de precios</li>
    <li>Haga clic en <b>Aplicar Filtros</b> para actualizar las visualizaciones</li>
    <li>Las visualizaciones son interactivas - puede hacer zoom, hover para ver detalles, etc.</li>
    <li>Use los botones de <b>Exportar</b> (Sección 7) para descargar los datos filtrados</li>
    <li>Para resetear los filtros, use el botón <b>Resetear Filtros</b></li>
</ol>

<h3>Nota:</h3>
<p>Para actualizar las visualizaciones después de aplicar filtros, vuelva a ejecutar las celdas de la Sección 5.</p>
</div>
"""))

print("\n✓ Dashboard listo para usar")